In [ ]:
#Second Model. 
#No Data Augmentation
#Hyperparameters tunning with Hyperas

In [4]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Conv2D
from keras.utils import np_utils
import h5py

from keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
def data():
    nb_classes = 1

    df = pd.read_csv('data.csv')
    df = df.drop('Unnamed: 0', axis=1)
    y= df['label']
    X= df.drop('label', axis=1)

    # Train/Test Split
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=142)
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    
    scaler = MinMaxScaler(feature_range = (0,1))
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    X_train = X_train.reshape(X_train.shape[0], 256, 150, 1)
    X_test = X_test.reshape(X_test.shape[0], 256, 150, 1)

    return X_train, Y_train, X_test, Y_test

In [6]:
def model(X_train, Y_train, X_test, Y_test):

    nb_epoch = 15

    # input image dimensions
    img_rows, img_cols = 256, 150
    img_channels = 1

    model = Sequential()

    model.add(Conv2D({{choice([90, 180])}}, (5,5), padding='same',
                            input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (4,4), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense({{choice([1024, 2048])}}))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes))
    model.add(Activation('relu'))

    adam = Adam()
    model.compile(loss='mean_absolute_error',
                  optimizer=adam, 
                  metrics=['mean_squared_error'])

    # fit the model on the batches generated by datagen.flow()
    model.fit(X_train, Y_train,
            batch_size=128,
            epochs=nb_epoch,
            validation_data=(X_test, Y_test))

    score, acc = model.evaluate(X_test, Y_test, verbose=0)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
%%timeit -n1 -r1

X_train, Y_train, X_test, Y_test = data()

best_run, best_model, space = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=2,
                                          trials=Trials(),
                                          eval_space=True,  
                                          return_space=True,
                                          notebook_name='CNN_model2')


X_train shape: (345, 38400)
345 train samples
148 test samples
>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.layers.convolutional import MaxPooling2D
except:
    pass

try:
    from keras.layers import Conv2D
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from sklearn.metrics import mean_squared_error, r2_s

In [ ]:
best_model.save("best_model2.h5")

In [ ]:
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))

In [ ]:
print("Parameters of best run", best_run)

In [ ]:
print(best_model.summary())

In [ ]:
preds_test = best_model.predict(X_test)
preds_train = best_model.predict(X_train)

In [ ]:
plt.scatter(preds, y_test, color='gray');
plt.xlabel('CNN Predicted Age (years)')
plt.plot([min(y_test),max(y_test)],[min(y_test),max(y_test)])
plt.ylabel('Age (years)')
plt.show()

In [ ]:
plt.scatter(preds_train, y_train, color='gray');
plt.xlabel('CNN Predicted Age (years)')
plt.ylabel('Age (years)')
plt.plot([min(y_train),max(y_train)],[min(y_train),max(y_train)])
plt.show()

In [ ]:
r_score_test = r2_score(Y_test, preds_test)
print (r_score_test)

In [ ]:
r_score_train = r2_score(Y_train, preds_train)
print (r_score_train)

In [2]:
import gc; gc.collect()

11